# 第6回 その2: 勾配降下法
勾配降下法を使ったパラメータの最適化をデモンストレーションします。

まずはライブラリのインポート。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## ステップ1: 二次関数の最小値
損失関数への入力パラメータを$x$（スカラー），損失関数を$L = (5 - x)^2$とします。  
このとき，損失関数を最小化する$x$は$x = 5$ですが，これを勾配降下法によって求めてみましょう。  

以下は損失関数を計算する関数です。

In [ ]:
def loss_function(x, a):
  '''
      x: 入力パラメータ
      a: 二次関数のパラメータ
  '''
  L = (a - x)**2
  return L

二次関数のパラメータを$a = 5$とします。  
$x$の初期値を $-5$とし，このときの損失値を計算してみます。

In [ ]:
# 損失関数のパラメータ
a = 5

# xの初期値
x_initial = -5

# 損失値
L = loss_function(x_initial, a)
print('loss = ' + str(L))

# 損失関数をプロットする
x_line = np.linspace(-10, 20) # -10から10に引かれたx軸
L_line = loss_function(x_line, a)
plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
plt.xlabel('parameter x')
plt.ylabel('loss function L')
# 現在の x と損失値をプロット
plt.scatter(x_initial, L, color='r')
plt.xlim([-10, 20])
plt.ylim([-5, 200])
plt.show()

さて，このときの勾配を計算します。  
$L = (a - x)^2$ の $x$ に対する勾配は  
$\frac{\partial L}{\partial x} = -2(a-x)$  
です。

In [ ]:
def calc_gradient(x, a):
  grad = -2.0 * (a - x)
  return grad

現在の$x$に対して勾配を計算します。  
また，計算された勾配を元に，損失関数上に接線を引きます。

In [ ]:
grad = calc_gradient(x_initial, a)
print('gradient of L for x_initial = ' + str(grad))

# 損失関数をプロットする
x_line = np.linspace(-10, 20) # -10から10に引かれたx軸
L_line = loss_function(x_line, a)
plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
plt.xlabel('parameter x')
plt.ylabel('loss function L')
# 現在の x と損失値をプロット
plt.scatter(x_initial, L, color='r')
# 接線を引く
tangent = grad * (x_line - x_initial) + L
plt.plot(x_line, tangent, color='red')
plt.xlim([-10, 20])
plt.ylim([-5, 200])
plt.show()

勾配は負の値になっていることが分かります。  
さて，計算された勾配を元に，勾配降下法を使って$x$を更新します。  
$x_{\rm next} = x - \mu \frac{\partial L}{\partial x}$  
ここでは，$\mu = 0.1$ とします。

In [ ]:
# 学習率(lr = learning rate)
lr = 0.1

# 勾配を使って w を更新
x_next = x_initial - lr * grad

print('x_next = ' + str(x_next))

勾配は負の値なので，$x$は正の方向へ修正されます。  
その結果，$x = -3.0$となりました。  
このときも同様に損失値と接線を求めてみます。  

In [ ]:
# 損失値
L_next = loss_function(x_next, a)
print('loss = ' + str(L))

# 勾配
grad_next = calc_gradient(x_next, a)
print('gradient of L for x_initial = ' + str(grad))


plt.figure(figsize=(6,6))
# 損失関数をプロット
plt.plot(x_line, L_line)
# 一つ前の x と損失値
plt.scatter(x_initial, L, color='r', label='previous x')
# 一つ前の接線
tangent = grad * (x_line - x_initial) + L
plt.plot(x_line, tangent, color='r')
# 更新後の x と損失値と接線
plt.scatter(x_next, L_next, color='g', label='next x')
tangent_next = grad_next * (x_line - x_next) + L_next
plt.plot(x_line, tangent_next, color='g')
plt.xlabel('parameter x')
plt.ylabel('loss function L')
plt.xlim([-10, 20])
plt.ylim([-5, 200])
plt.legend()
plt.show()

更新によって$x$は最適値に近づき，それに伴って損失値が小さくなり，また勾配も0に近づきました。  

以降，上記の処理を繰り返します。

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

# 学習率(lr = learning rate)
lr = 0.1

# 更新回数
num_iterations = 30

# 初期値をコピー
x = x_initial

# 描画
fig = plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
images = []

for n in range(num_iterations):
    # 損失値を計算
    L = loss_function(x, a)
    # 勾配を計算
    grad = calc_gradient(x, a)

    print("%d-th iteration, x=%.3f, loss: %.3f, grad: %.3f" % (n, x, L, grad))

    # 更新前の状態を描画
    tangent = grad * (x_line - x) + L
    img = plt.plot(x_line, tangent, color='r')
    img.append(plt.scatter(x, L, color='r'))
    img.append(plt.text(-8, 180, 'iteration: '+str(n), size='x-large'))
    images.append(img)
    
    # 更新
    x = x - lr * grad

plt.xlim([-10, 20])
plt.ylim([-5, 200])
plt.xlabel('parameter x')
plt.ylabel('loss function L')

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=100)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)


更新を繰り返すことで，$x$が徐々に$a=5$の値に近づき，それに伴って損失関数が最小値に，勾配が0に近づいていることが分かります。

## ステップ2: 学習率による挙動の違い
学習率$\mu$が小さいと，更新の度合いが小さくなるため，収束は遅くなります。  
一方，学習率を大きくすると，更新の度合いは大きくなりますが，$x$が最適値の前後を行ったり来たりする，**振動状態**が起こる場合があります。  
ためしに学習率を0.9にして，上の処理を再実行してみます。

In [ ]:
# 学習率(lr = learning rate)
lr = 0.9

# 更新回数
num_iterations = 30

# 初期値をコピー
x = x_initial

# 描画
fig = plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
images = []

for n in range(num_iterations):
    # 損失値を計算
    L = loss_function(x, a)
    # 勾配を計算
    grad = calc_gradient(x, a)

    print("%d-th iteration, x=%.3f, loss: %.3f, grad: %.3f" % (n, x, L, grad))

    # 更新前の状態を描画
    tangent = grad * (x_line - x) + L
    img = plt.plot(x_line, tangent, color='r')
    img.append(plt.scatter(x, L, color='r'))
    img.append(plt.text(-8, 180, 'iteration: '+str(n), size='x-large'))
    images.append(img)
    
    # 更新
    x = x - lr * grad

plt.xlim([-10, 20])
plt.ylim([-5, 200])
plt.xlabel('parameter x')
plt.ylabel('loss function L')

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=100)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)


$x$が5の前後で振動しており，収束が遅くなっていることが確認できます。  
ちなみに，上の例において学習率を1.0以上にすると，振動が大きくなりすぎて収束しなくなります。  
（lrを1.0以上にして，挙動を確認してみてください。）  
ですので，上記の例においては，学習率はある程度小さい方が良いということになります。  

## ステップ3: 局所最適解の問題  
上の例のように損失関数が二次関数の場合，うまく学習率を設定しておけば最適解にたどり着くことができます。  
しかし極小値が複数あるような関数の場合，最適解にたどり着けないことがあります。  

例として，以下のような損失関数を考えてみます。


In [ ]:
def loss_function2(x):
  '''
      x: 入力パラメータ
      a: 二次関数のパラメータ
  '''
  L = np.sin(x) + 0.05*(5 -x)**2
  return L

この式の勾配は以下のように定義されます。  

In [ ]:
def calc_gradient2(x):
  grad = np.cos(x) - 0.10*(5 -x)
  return grad

$x$の初期値を$x = -10$として，プロットしてみます。

In [ ]:
# xの初期値
x_initial = -10

# 損失値
L = loss_function2(x_initial)
print('loss = ' + str(L))

# 損失関数をプロットする
x_line = np.linspace(-20, 22) # -10から10に引かれたx軸
L_line = loss_function2(x_line)
plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
plt.xlabel('parameter x')
plt.ylabel('loss function L')
# 現在の x と損失値をプロット
plt.scatter(x_initial, L, color='r')
plt.xlim([-12, 22])
plt.ylim([-2.5, 17])
plt.show()

ではこの損失関数のもとで，学習率を1.0として勾配降下法を実施してみましょう。

In [ ]:
# 学習率(lr = learning rate)
lr = 1.0

# 更新回数
num_iterations = 30

# 初期値をコピー
x = x_initial

# 描画
fig = plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
images = []

for n in range(num_iterations):
    # 損失値を計算
    L = loss_function2(x)
    # 勾配を計算
    grad = calc_gradient2(x)

    print("%d-th iteration, x=%.3f, loss: %.3f, grad: %.3f" % (n, x, L, grad))

    # 更新前の状態を描画
    #img1 = plt.plot(x, L, c='r', marker='o', markersize=8)
    tangent = grad * (x_line - x) + L
    img = plt.plot(x_line, tangent, color='r')
    img.append(plt.scatter(x, L, color='r'))
    img.append(plt.text(-10, 16, 'iteration: '+str(n), size='x-large'))
    images.append(img)
    
    # 更新
    x = x - lr * grad

plt.xlim([-12, 22])
plt.ylim([-2.5, 17])
plt.xlabel('parameter x')
plt.ylabel('loss function L')

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=100)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)


損失関数の極小値に引っかかってしまい，最小値に到達できていないことが分かります。  
このように，複数存在する極小値のことを，<font color='red'>**局所最適解（局所解）**</font>と呼びます。  


一般に，学習率が小さいと局所最適解に陥りやすいです。  
では学習率を大きくするとどうでしょうか？  
学習率を3.0にして，再度実行してみましょう。

In [ ]:
# 学習率(lr = learning rate)
lr = 3.0

# 更新回数
num_iterations = 30

# 初期値をコピー
x = x_initial

# 描画
fig = plt.figure(figsize=(6,6))
plt.plot(x_line, L_line)
images = []

for n in range(num_iterations):
    # 損失値を計算
    L = loss_function2(x)
    # 勾配を計算
    grad = calc_gradient2(x)

    print("%d-th iteration, x=%.3f, loss: %.3f, grad: %.3f" % (n, x, L, grad))

    # 更新前の状態を描画
    #img1 = plt.plot(x, L, c='r', marker='o', markersize=8)
    tangent = grad * (x_line - x) + L
    img = plt.plot(x_line, tangent, color='r')
    img.append(plt.scatter(x, L, color='r'))
    img.append(plt.text(-10, 16, 'iteration: '+str(n), size='x-large'))
    images.append(img)
    
    # 更新
    x = x - lr * grad

plt.xlim([-12, 22])
plt.ylim([-2.5, 17])
plt.xlabel('parameter x')
plt.ylabel('loss function L')

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=100)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)


局所最適解に引っかかることはなくなりましたが，今度は振動状態が発生し，結局最適解にたどり着けませんでした。  
このように，局所最適解は難しい問題となっています。